In [5]:
import sys
import importlib

if "local_InteractingHPSN" in sys.modules:
    del sys.modules["local_InteractingHPSN"]
import local_InteractingHPSN
importlib.reload(local_InteractingHPSN)

print(local_InteractingHPSN.__file__)

from local_InteractingHPSN import (
    InteractingHPSN_LC,
    generateInteractingHPSNSlicer,
    BaseInteractingHPSNMetric,
    InteractingHPSN_DetectMetric,
    InteractingHPSN_ClassicalMetric,
    InteractingHPSN_AmbiguousMetric,
    InteractingHPSN_UncharacterizedMetric,
    InteractingHPSN_SeparabilityMetric,
    InteractingHPSN_TriggerableMetric
)

print(local_InteractingHPSN.__file__)


/Users/andradenebula/Documents/Research/Transient_Metrics/Stuff/AllTransient/local_InteractingHPSN.py
/Users/andradenebula/Documents/Research/Transient_Metrics/Stuff/AllTransient/local_InteractingHPSN.py


In [7]:
# -------------------------------
# Configuration
# -------------------------------
db_file = "baseline_v4.3.1_10yrs.db"
run_name = os.path.splitext(os.path.basename(db_file))[0]
out_dir = f"output_{run_name}_InteractingHPSN"
os.makedirs(out_dir, exist_ok=True)

# -------------------------------
# Load LSST footprint probability map
# -------------------------------
opsdb = OpsimDatabase(db_file)
metric = CountMetric(col='night')
slicer = hp.HealpixSlicer(nside=64)
maps = [StellarDensityMap(nside=64)]
foot_bundle = MetricBundle(metric, slicer, maps=maps)
foot_group = MetricBundleGroup({"footprint": foot_bundle}, opsdb, out_dir=out_dir)
foot_group.runAll()
prob_map = foot_bundle.metricValues
prob_map /= np.sum(prob_map)  # normalize

# -------------------------------
# Generate SN population and slicer
# -------------------------------
template_path = os.path.join(out_dir, "InteractingHPSN_templates.pkl")
pop_path = os.path.join(out_dir, "InteractingHPSN_population.pkl")

lc_model = InteractingHPSN_LC(save_to=template_path)
slicer = generateInteractingHPSNSlicer(prob_map, lc_model, save_to=pop_path)

# -------------------------------
# Define metrics
# -------------------------------
metrics = {
    "InteractingHPSN_Detect": InteractingHPSN_DetectMetric(),
    "InteractingHPSN_Classical": InteractingHPSN_ClassicalMetric(),
    "InteractingHPSN_Ambiguous": InteractingHPSN_AmbiguousMetric(),
    "InteractingHPSN_Uncharacterized": InteractingHPSN_UncharacterizedMetric(),
    "InteractingHPSN_Separability": InteractingHPSN_SeparabilityMetric(),
    "InteractingHPSN_Triggerable": InteractingHPSN_TriggerableMetric()
}

# -------------------------------
# Run metrics
# -------------------------------
bundles = {}
for name, metric in metrics.items():
    bundles[name] = MetricBundle(metric, slicer, maps=[], metricName=name)

group = MetricBundleGroup(bundles, opsdb, out_dir=out_dir)
group.runAll()

# -------------------------------
# Save results
# -------------------------------
print(f"Metrics complete. Results in: {out_dir}")


AttributeError: module 'healpy' has no attribute 'HealpixSlicer'